In [11]:
import cv2
import numpy as np
import os

In [12]:
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

In [13]:
#implement KNN
def KNN(train,test,k=5):
    m=train.shape[0]
    vals=[]
    for i in range(m):
        ix=train[i,:-1]
        iy=train[i,-1]
        d=dist(test,ix)
        vals.append([d,iy])
        
    vals=sorted(vals,key=lambda x:x[0])[:k]
    #vals=vals[:k]
    vals=np.array(vals)[:,-1]
    vals=np.unique(vals,return_counts=True)
    print(vals)
    index=vals[1].argmax()
    predict=vals[0][index]
    #print(vals)
    return predict

In [14]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
dataset_path='./data/'
face_data=[]
labels=[]
class_id=0
names={}

In [15]:
#load files and store files into face_data and create corresponds labels
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        dataitem=np.load(dataset_path+fx)
        face_data.append(dataitem)
        names[class_id]=fx[:-4]
        target=class_id*np.ones((dataitem.shape[0],))
        class_id=class_id+1
        labels.append(target)

In [16]:
#concate face_data that is list of list into a single matrix
trainX=np.concatenate(face_data,axis=0)
#concate labels that is list of list into a single matrix that have single column
trainY=np.concatenate(labels,axis=0).reshape((-1,1))

In [17]:
print(trainX.shape)
print(trainY.shape)

(109, 30000)
(109, 1)


In [18]:
#concatenate the trainX and trainY into a single matrix
training_data=np.concatenate((trainX,trainY),axis=1)

In [19]:
#Now generate the testing data
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    for (x,y,w,h) in faces:
        face_section=frame[y-20:y+h+20,x-20:x+w+20]
        face_section=cv2.resize(face_section,(100,100))
        #now predict the result on testing data
        out=KNN(training_data,face_section.flatten())
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("faces",frame)
    key=cv2.waitKey(1)&0xFF
    if key==ord('q'):
        break

(array([1., 3.]), array([4, 1], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([2., 3.]), array([1, 4], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([3.]), array([5], dtype=int64))
(array([0.]), array([5], dtype=int64))
(array([0.]), array([5], dtype=int64))
(array([0.]

In [20]:
cap.release()
cv2.destroyAllWindows()